In [24]:
import cv2
import numpy as np
import webbrowser

# YOLO 가중치 파일, 구성 파일, 클래스 이름 파일 경로
weights_path = "yolov3.weights"
config_path = "yolov3.cfg"
names_path = "coco.names"

# YOLO 모델 클래스 이름 로드
with open(names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# 네트워크 로드
net = cv2.dnn.readNet(weights_path, config_path)

# 동영상 파일 경로
video_path = "refrigerator.mp4"

# 동영상 읽기
cap = cv2.VideoCapture(video_path)

# 클릭된 객체 식재료
clicked_item = None

# 마우스 클릭 이벤트 핸들러
def click_event(event, x, y, flags, param):
    global clicked_item
    if event == cv2.EVENT_LBUTTONDOWN:
        for i in range(len(boxes)):
            if i in indexes:
                x1, y1, w1, h1 = boxes[i]
                if x1 < x < x1 + w1 and y1 < y < y1 + h1:
                    clicked_item = classes[class_ids[i]]
                    search_recipe(clicked_item)
                    break

# 레시피 검색
def search_recipe(item):
    query = f"{item} 레시피"
    url = f"https://www.google.com/search?q={query}"
    webbrowser.open(url)

# 창 이름
window_name = "Refrigerator Content"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# 동영상 재생
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    
    # 블롭(blob) 생성 및 네트워크에 입력
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # 출력 레이어 이름 가져오기
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()

    # unconnected_out_layers가 배열이 아닌 경우 처리
    if isinstance(unconnected_out_layers, (np.ndarray, list)):
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers]
    else:
        output_layers = [layer_names[unconnected_out_layers - 1]]

    # 객체 탐지
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # 탐지된 객체 정보 추출
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # 노이즈 제거를 위한 Non-max Suppression 적용
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # 탐지된 객체 박스 및 레이블 그리기
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # green
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 동영상 출력
    cv2.imshow(window_name, frame)

    # 클릭 이벤트 설정
    cv2.setMouseCallback(window_name, click_event)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()